# World Cup 2022 winner classifier
Imagine being able to predict which team will win during a match. All your bets would win and you'd be rich! Unfortunately, football, like any sport, is unpredictable, especially at the World Cup.


An attempt to build such an ML model is doomed to failure, the only question is how much? The project's task is to predict the winner of a given match, and thanks to the ongoing championships, I can check the accuracy of the model.

Datasets used: 
FIFA ranking -  https://www.kaggle.com/code/agostontorok/soccer-world-cup-2018-winner/data
football results -  https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

In [2]:
#importing data
fifa_ranking = pd.read_csv(r"data\world cup 2022\fifa_ranking-2022-10-06.csv")
matches =  pd.read_csv(r"data\world cup 2022\results.csv")

## Matches data 

In [3]:
matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44201,2022-12-01,Canada,Morocco,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44202,2022-12-02,Serbia,Switzerland,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44203,2022-12-02,Cameroon,Brazil,NaN,NaN,FIFA World Cup,Lusail,Qatar,True
44204,2022-12-02,Ghana,Uruguay,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True


In [4]:
#renaming columns 
matches.rename(columns = {'home_team':'team_1', 'away_team':'team_2','home_score':'team_1_score', 'away_score':'team_2_score'}, inplace = True)

In [5]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          44206 non-null  object 
 1   team_1        44206 non-null  object 
 2   team_2        44206 non-null  object 
 3   team_1_score  44186 non-null  float64
 4   team_2_score  44186 non-null  float64
 5   tournament    44206 non-null  object 
 6   city          44206 non-null  object 
 7   country       44206 non-null  object 
 8   neutral       44206 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [6]:
# date dtype is object, let's convert it to datetime
matches["date"] = pd.to_datetime(matches["date"], format = '%Y-%m-%d')

In [238]:
# these matches have not yet taken place or have taken place but on current world cup, they need to be deleted
matches.tail(20)

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral
44186,2022-11-28,Cameroon,Serbia,NaN,NaN,FIFA World Cup,Al Wakrah,Qatar,True
44187,2022-11-28,Brazil,Switzerland,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44188,2022-11-28,South Korea,Ghana,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44189,2022-11-28,Portugal,Uruguay,NaN,NaN,FIFA World Cup,Lusail,Qatar,True
44190,2022-11-29,Ecuador,Senegal,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44191,2022-11-29,Qatar,Netherlands,NaN,NaN,FIFA World Cup,Al Khor,Qatar,False
44192,2022-11-29,Wales,England,NaN,NaN,FIFA World Cup,Al Rayyan,Qatar,True
44193,2022-11-29,Iran,United States,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44194,2022-11-30,Poland,Argentina,NaN,NaN,FIFA World Cup,Doha,Qatar,True
44195,2022-11-30,Saudi Arabia,Mexico,NaN,NaN,FIFA World Cup,Lusail,Qatar,True


In [239]:
matches.drop(matches.index[44158:], axis=0, inplace=True )
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True


## Ranking data

In [240]:
fifa_ranking

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
1,96,Syria,SYR,11.00,0.00,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.00,0.00,0,CAF,1992-12-31
3,99,Latvia,LVA,10.00,0.00,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...
63911,74,El Salvador,SLV,1330.51,1333.48,3,CONCACAF,2022-10-06
63912,75,Oman,OMA,1320.29,1323.03,0,AFC,2022-10-06
63913,76,Israel,ISR,1316.55,1316.35,0,UEFA,2022-10-06
63914,78,Georgia,GEO,1307.34,1296.46,-4,UEFA,2022-10-06


In [241]:
#renaming columns 
fifa_ranking.rename(columns = {'rank_date':'date'}, inplace = True)

In [242]:
fifa_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             63916 non-null  int64  
 1   country_full     63916 non-null  object 
 2   country_abrv     63916 non-null  object 
 3   total_points     63916 non-null  float64
 4   previous_points  63916 non-null  float64
 5   rank_change      63916 non-null  int64  
 6   confederation    63916 non-null  object 
 7   date             63916 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ MB


In [243]:
# date dtype is object, let's convert it to datetime
fifa_ranking["date"] = pd.to_datetime(fifa_ranking["date"], format = '%Y-%m-%d')

## Final data set

In [244]:
#making target variable
matches["result"] = 1
matches.loc[matches.team_1_score > matches.team_2_score, "result" ] = 0
matches.loc[matches.team_1_score < matches.team_2_score, "result" ] = 2

In [245]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,0
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,0
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,0
...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2


In [246]:
#the world cup is on a neutral ground, so we should remove the other options
# However, it is worth keeping the data. We will see later if this was good choice.

In [247]:
#the first ranking date appeared in 1993, because of that we have to drop matches before this date
fifa_ranking.sort_values("date")

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
95,74,Madagascar,MAD,18.00,0.00,0,CAF,1992-12-31
96,2,Italy,ITA,57.00,0.00,0,UEFA,1992-12-31
97,3,Brazil,BRA,56.00,0.00,0,CONMEBOL,1992-12-31
98,4,Sweden,SWE,56.00,0.00,0,UEFA,1992-12-31
...,...,...,...,...,...,...,...,...
63778,209,British Virgin Islands,VGB,809.32,809.32,0,CONCACAF,2022-10-06
63779,107,Kosovo,KOS,1187.01,1183.90,1,UEFA,2022-10-06
63780,200,Eritrea,ERI,855.56,855.56,-1,CAF,2022-10-06
63782,186,São Tomé and Príncipe,STP,908.18,908.18,-1,CAF,2022-10-06


In [248]:
first_date = fifa_ranking.sort_values("date").iloc[0].date
matches = matches[matches.date >= first_date]
first_date

Timestamp('1992-12-31 00:00:00')

In [249]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result
17697,1993-01-01,Ghana,Mali,1.0,1.0,Friendly,Libreville,Gabon,True,1
17698,1993-01-02,Gabon,Burkina Faso,1.0,1.0,Friendly,Libreville,Gabon,False,1
17699,1993-01-02,Kuwait,Lebanon,2.0,0.0,Friendly,Kuwait City,Kuwait,False,0
17700,1993-01-03,Burkina Faso,Mali,1.0,0.0,Friendly,Libreville,Gabon,True,0
17701,1993-01-03,Gabon,Ghana,2.0,3.0,Friendly,Libreville,Gabon,False,2
...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2


In [250]:
#writing a function to assign the appropriate index from the fifa_ranking df, where the team's rank is located
def add_ranking_index(team1, team2, date):
    ranking = fifa_ranking[fifa_ranking.date <= date]
    rank_team1 = ranking[ranking.country_full == team1]
    rank_team2 = ranking[ranking.country_full == team2]
    
    if(rank_team1.empty):
        team1_index = np.NaN
    else:
        team1_index = rank_team1.sort_values("date", ascending=False).index[0]
        
    if(rank_team2.empty):
        team2_index = np.NaN
    else:
        team2_index = rank_team2.sort_values("date", ascending=False).index[0]
        
    return pd.Series([team1_index, team2_index])

In [251]:
#it seems to be working
add_ranking_index(matches.iloc[-1].team_1, matches.iloc[-1].team_2, matches.iloc[-1].date)

0    63890
1    63816
dtype: int64

In [255]:
#scale it up 
matches[['id1', 'id2']] = matches.apply(lambda row: add_ranking_index(row["team_1"], row["team_2"], row["date"]), axis=1)

In [256]:
matches

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result,id1,id2
17697,1993-01-01,Ghana,Mali,1.0,1.0,Friendly,Libreville,Gabon,True,1,132.0,125.0
17698,1993-01-02,Gabon,Burkina Faso,1.0,1.0,Friendly,Libreville,Gabon,False,1,130.0,2.0
17699,1993-01-02,Kuwait,Lebanon,2.0,0.0,Friendly,Kuwait City,Kuwait,False,0,127.0,NaN
17700,1993-01-03,Burkina Faso,Mali,1.0,0.0,Friendly,Libreville,Gabon,True,0,2.0,125.0
17701,1993-01-03,Gabon,Ghana,2.0,3.0,Friendly,Libreville,Gabon,False,2,130.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...
44153,2022-11-20,Slovenia,Montenegro,1.0,0.0,Friendly,Ljubljana,Slovenia,False,0,63900.0,63906.0
44154,2022-11-20,South Africa,Angola,1.0,1.0,Friendly,Mbombela,South Africa,False,1,63904.0,63742.0
44155,2022-11-20,Saint Lucia,San Marino,1.0,0.0,Friendly,Gros Islet,Saint Lucia,False,0,NaN,63915.0
44156,2022-11-20,Syria,Venezuela,1.0,2.0,Friendly,Dubai,United Arab Emirates,True,2,63873.0,63894.0


In [257]:
#some of the values are NaN, let's see why
matches.loc[matches.isnull().values, ["date", "team_1", "team_2", "id1", "id2" ]]
#looks like first nan values are due to the lack of country in ranking (future rankings include more countries)

,date,team_1,team_2,id1,id2
17699,1993-01-02,Kuwait,Lebanon,127.0,NaN
17706,1993-01-10,DR Congo,Cameroon,NaN,77.0
17713,1993-01-17,Botswana,Ivory Coast,41.0,NaN
17717,1993-01-17,Namibia,Madagascar,NaN,95.0
17721,1993-01-19,North Korea,Bolivia,NaN,29.0
...,...,...,...,...,...
44087,2022-11-16,Iran,Tunisia,NaN,63813.0
44088,2022-11-16,Ivory Coast,Burundi,NaN,63706.0
44119,2022-11-17,Saint Lucia,San Marino,NaN,63915.0
44131,2022-11-19,Ivory Coast,Burkina Faso,NaN,63891.0


In [258]:
#but what about other nan values? Maybe problem is with country name?
matches.loc[matches.id1.isnull().values, "team_1"].value_counts()[:10]
# matches.loc[matches.id2.isnull().values, "team_2"].value_counts()[:10]

United States            338
South Korea              269
Iran                     202
Ivory Coast              162
DR Congo                 127
Saint Kitts and Nevis     93
Martinique                91
North Korea               80
North Macedonia           72
Guadeloupe                71
Name: team_1, dtype: int64

In [259]:
np.sort(fifa_ranking.country_full.unique())

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde Islands', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China PR',
       'Chinese Taipei', 'Colombia', 'Comoros', 'Congo', 'Congo DR',
       'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curacao',
       'Curaçao', 'Cyprus', 'Czech Republic', 'Czechoslovakia',
       "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England',
       'Equato

In [210]:
#United States is USA in FIFA
#South Korea is Korea Republic
#Iran is IR Iran
#Ivory Coast is Côte d'Ivoire
#North Macedonia is  FYR Macedonia
#DR Congo  is Congo or Congo DR
#North Korea is Korea DPR
#Northern Cyprus is Cyprus
#Eswatini is Swaziland
#Ireland is Republic of Ireland
#rest are insignificant teams and I will drop them

In [260]:
#replacing all wrong names to FIFA names 
matches.team_1.replace(['United States'], ['USA'], inplace = True)
matches.team_2.replace(['United States'], ['USA'], inplace = True)

matches.team_1.replace(['South Korea'], ['Korea Republic'], inplace = True)
matches.team_2.replace(['South Korea'], ['Korea Republic'], inplace = True)

matches.team_1.replace(['Iran'], ['IR Iran'], inplace = True)
matches.team_2.replace(['Iran'], ['IR Iran'], inplace = True)

matches.team_1.replace(['Ivory Coast'], ["Côte d'Ivoire"], inplace = True)
matches.team_2.replace(['Ivory Coast'], ["Côte d'Ivoire"], inplace = True)

matches.team_1.replace(['North Macedonia'], ['FYR Macedonia'], inplace = True)
matches.team_2.replace(['North Macedonia'], ['FYR Macedonia'], inplace = True)

matches.team_1.replace(['DR Congo'], ['Congo'], inplace = True)
matches.team_2.replace(['DR Congo'], ['Congo'], inplace = True)

fifa_ranking.country_full.replace(['Congo DR'], ['Congo'], inplace = True)

matches.team_1.replace(['North Korea'], ['Korea DPR'], inplace = True)
matches.team_2.replace(['North Korea'], ['Korea DPR'], inplace = True)

matches.team_1.replace(['Northern Cyprus'], ['Cyprus'], inplace = True)
matches.team_2.replace(['Northern Cyprus'], ['Cyprus'], inplace = True)

matches.team_1.replace(['Eswatini'], ['Swaziland'], inplace = True)
matches.team_2.replace(['Eswatini'], ['Swaziland'], inplace = True)

matches.team_1.replace(['Ireland'], ['Republic of Ireland'], inplace = True)
matches.team_2.replace(['Ireland'], ['Republic of Ireland'], inplace = True)




In [261]:
#now make it with correct names
matches[["id1", "id2"]] = matches.apply(lambda row: assign_rank(row["team_1"], row["team_2"], row["date"]), axis=1)

In [262]:
matches.loc[matches.id1.isnull().values, "team_1"].value_counts()[:]

Saint Kitts and Nevis        93
Martinique                   91
Guadeloupe                   71
Cape Verde                   68
Saint Lucia                  63
                             ..
Madrid                        1
Sark                          1
Saint Pierre and Miquelon     1
Silesia                       1
Aymara                        1
Name: team_1, Length: 140, dtype: int64

In [263]:
#muuuuch better :)
matches.loc[matches.id2.isnull().values, "team_2"].value_counts()[:]

Kyrgyzstan       88
Martinique       87
Saint Lucia      82
Serbia           80
Cape Verde       78
                 ..
FYR Macedonia     1
Brittany          1
Kazakhstan        1
Armenia           1
Aymara            1
Name: team_2, Length: 142, dtype: int64

In [7]:
#saving matches to avoid recalculation
# matches.to_csv(r".\data\world cup 2022\matches_with_rank.csv", index=False)
matches_rank = pd.read_csv(r".\data\world cup 2022\matches_with_rank.csv")

In [8]:
#droping all rows where is NaN id
matches_rank.dropna(axis=0, inplace=True)

In [9]:
#know it is time to merge this datasets on id
matches_merged = matches_rank.merge(fifa_ranking, how = 'inner', left_on = 'id1', right_index = True, suffixes = ['', '_1'])
matches_merged = matches_merged.merge(fifa_ranking, how = 'inner', left_on = 'id2', right_index = True, suffixes = ['', '_2'])

In [10]:
matches_merged

,date,team_1,team_2,team_1_score,team_2_score,tournament,city,country,neutral,result,...,confederation,rank_date,rank_2,country_full_2,country_abrv_2,total_points_2,previous_points_2,rank_change_2,confederation_2,rank_date_2
0,1993-01-01,Ghana,Mali,1.0,1.0,Friendly,Libreville,Gabon,True,1,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
475,1993-07-04,Gabon,Mali,0.0,2.0,Friendly,Moanda,Gabon,False,2,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
3,1993-01-03,Burkina Faso,Mali,1.0,0.0,Friendly,Libreville,Gabon,True,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
171,1993-04-09,Egypt,Mali,2.0,1.0,African Cup of Nations qualification,Cairo,Egypt,False,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
246,1993-04-25,Morocco,Mali,1.0,0.0,African Cup of Nations qualification,Casablanca,Morocco,False,0,...,CAF,1992-12-31,69,Mali,MLI,22.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26423,2022-11-17,Switzerland,Ghana,0.0,2.0,Friendly,Abu Dhabi,United Arab Emirates,True,2,...,UEFA,2022-10-06,61,Ghana,GHA,1393.00,1393.47,1,CAF,2022-10-06
26429,2022-11-18,Egypt,Belgium,2.0,1.0,Friendly,Kuwait City,Kuwait,True,0,...,CAF,2022-10-06,2,Belgium,BEL,1816.71,1821.92,0,UEFA,2022-10-06
26436,2022-11-19,Kuwait,Lebanon,2.0,0.0,Friendly,Dubai,United Arab Emirates,True,0,...,AFC,2022-10-06,99,Lebanon,LBN,1211.71,1211.71,-1,AFC,2022-10-06
26442,2022-11-19,Estonia,Lithuania,2.0,0.0,Baltic Cup,Tallinn,Estonia,False,0,...,UEFA,2022-10-06,144,Lithuania,LTU,1069.75,1074.08,2,UEFA,2022-10-06


In [11]:
#changing names for convenience
matches_merged.columns = ['date', 'team_1', 'team_2', 'team_1_score', 'team_2_score',
       'tournament', 'city', 'country', 'neutral', 'result', 'id1', 'id2',
       'rank_1', 'country_full_1', 'country_abrv_1', 'total_points_1',
       'previous_points_1', 'rank_change_1', 'confederation_1', 'date_1', 'rank_2',
       'country_full_2', 'country_abrv_2', 'total_points_2',
       'previous_points_2', 'rank_change_2', 'confederation_2', 'date_2']

In [12]:
#droping useless columns
matches_merged.drop(['team_1', 'team_2', 'team_1_score', 'team_2_score', 'city',
              'country', 'id1', 'id2', 'country_full_1',
              'country_abrv_1', 'date_1', 'country_full_2',
              'country_abrv_2', 'date_2'], axis = 1, inplace = True)

In [40]:
matches_merged

,date,tournament,neutral,result,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta
0,1993-01-01,Friendly,True,1,39,34.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318
475,1993-07-04,Friendly,False,2,55,27.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-353.025729
3,1993-01-03,Friendly,True,0,97,11.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318
171,1993-04-09,African Cup of Nations qualification,False,0,21,45.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-356.048379
246,1993-04-25,African Cup of Nations qualification,False,0,41,32.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-355.029877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26423,2022-11-17,Friendly,True,2,15,1635.92,1621.43,-1,UEFA,61,1393.00,1393.47,1,CAF,-1.018501
26429,2022-11-18,Friendly,True,0,39,1486.44,1482.63,-1,CAF,2,1816.71,1821.92,0,UEFA,-1.018501
26436,2022-11-19,Friendly,True,0,149,1053.41,1053.41,0,AFC,99,1211.71,1211.71,-1,AFC,-1.018501
26442,2022-11-19,Baltic Cup,False,0,109,1184.64,1177.40,-1,UEFA,144,1069.75,1074.08,2,UEFA,-1.018501


## Preprocessing

In [43]:
#some feature engineering

#time delta since the last date in the set
matches_merged["date"] = pd.to_datetime(matches_merged["date"], format = '%Y-%m-%d')
matches_merged["time_delta"] = (matches_merged["date"].values - np.datetime64('2022-11-20')).astype('timedelta64[M]')
matches_merged["time_delta"] = matches_merged["time_delta"].apply(lambda x: x / np.timedelta64(1,'M'))

#year
matches_merged['year'] = matches_merged["date"].dt.year
#month
matches_merged['month'] = matches_merged["date"].dt.month
#day of week
matches_merged['year'] = matches_merged["date"].dt.dayofweek
#day of year
matches_merged['day_of_year'] = matches_merged["date"].dt.dayofyear

In [44]:
matches_merged

,date,tournament,neutral,result,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_year
0,1993-01-01,Friendly,True,1,39,34.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,4,1,1
475,1993-07-04,Friendly,False,2,55,27.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-353.025729,6,7,185
3,1993-01-03,Friendly,True,0,97,11.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,6,1,3
171,1993-04-09,African Cup of Nations qualification,False,0,21,45.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-356.048379,4,4,99
246,1993-04-25,African Cup of Nations qualification,False,0,41,32.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-355.029877,6,4,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26423,2022-11-17,Friendly,True,2,15,1635.92,1621.43,-1,UEFA,61,1393.00,1393.47,1,CAF,-1.018501,3,11,321
26429,2022-11-18,Friendly,True,0,39,1486.44,1482.63,-1,CAF,2,1816.71,1821.92,0,UEFA,-1.018501,4,11,322
26436,2022-11-19,Friendly,True,0,149,1053.41,1053.41,0,AFC,99,1211.71,1211.71,-1,AFC,-1.018501,5,11,323
26442,2022-11-19,Baltic Cup,False,0,109,1184.64,1177.40,-1,UEFA,144,1069.75,1074.08,2,UEFA,-1.018501,5,11,323


In [47]:
#now I can delete data feature, everything was extracted :)
matches_merged.drop("date", axis = 1, inplace = True)

In [91]:
#time to label encoding
#quick look on tournament feature.Well...that's many categories. To many to just One Hot Encoding
#maybe good idea is to reduce number of different categories
matches_merged.tournament.value_counts()

Friendly                                8557
FIFA World Cup qualification            5486
UEFA Euro qualification                 1721
African Cup of Nations qualification    1272
AFC Asian Cup qualification              496
                                        ... 
CONMEBOL–UEFA Cup of Champions             2
Copa Confraternidad                        1
ELF Cup                                    1
TIFOCO Tournament                          1
Three Nations Cup                          1
Name: tournament, Length: 84, dtype: int64

In [108]:
#let's One hot encode all tournaments where number of matches > 100 and all others put in one "other" category
counts = matches_merged.tournament.value_counts()
def tournament_groupby(tourn, value):
    if counts[tourn] <= value:
        return "other"
    else:
        return tourn

In [118]:
matches_merged["tournament"] = matches_merged["tournament"].apply(lambda row: tournament_groupby(row, 100))

In [129]:
#much better :)
matches_merged.tournament.value_counts()

Friendly                                8557
FIFA World Cup qualification            5486
other                                   1752
UEFA Euro qualification                 1721
African Cup of Nations qualification    1272
AFC Asian Cup qualification              496
African Cup of Nations                   491
UEFA Nations League                      468
FIFA World Cup                           432
Copa América                             320
CECAFA Cup                               308
Gold Cup                                 299
COSAFA Cup                               293
AFF Championship                         258
UEFA Euro                                253
AFC Asian Cup                            227
African Nations Championship             223
Gulf Cup                                 204
CFU Caribbean Cup qualification          177
UNCAF Cup                                151
SAFF Cup                                 144
Confederations Cup                       136
Name: tour

In [120]:
#and let's see confederation_1 column
matches_merged.confederation_1.value_counts()
#not bad :)

UEFA        7739
CAF         5731
AFC         5432
CONCACAF    2752
CONMEBOL    1638
OFC          376
Name: confederation_1, dtype: int64

In [130]:
#time to encoding with pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

matches_merged['neutral'].replace([False, True], [0, 1], inplace = True)

encoder_pipe = ColumnTransformer(transformers=[
    ("enc", OneHotEncoder(), ["tournament", "confederation_1", "confederation_2"])], remainder = 'passthrough')

In [139]:
encoder_pipe.fit_transform(matches_merged)[0]

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    1.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    1.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    1.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    1.        ,    1.        ,
         39.        ,   34.        ,    0.        ,    0.        ,
         69.        ,   22.        ,    0.        ,    0.        ,
       -359.00531838,    4.        ,    1.        ,    1.        ])

In [140]:
matches_merged

,tournament,neutral,result,rank_1,total_points_1,previous_points_1,rank_change_1,confederation_1,rank_2,total_points_2,previous_points_2,rank_change_2,confederation_2,time_delta,year,month,day_of_year
0,Friendly,1,1,39,34.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,4,1,1
475,Friendly,0,2,55,27.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-353.025729,6,7,185
3,Friendly,1,0,97,11.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-359.005318,6,1,3
171,African Cup of Nations qualification,0,0,21,45.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-356.048379,4,4,99
246,African Cup of Nations qualification,0,0,41,32.00,0.00,0,CAF,69,22.00,0.00,0,CAF,-355.029877,6,4,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26423,Friendly,1,2,15,1635.92,1621.43,-1,UEFA,61,1393.00,1393.47,1,CAF,-1.018501,3,11,321
26429,Friendly,1,0,39,1486.44,1482.63,-1,CAF,2,1816.71,1821.92,0,UEFA,-1.018501,4,11,322
26436,Friendly,1,0,149,1053.41,1053.41,0,AFC,99,1211.71,1211.71,-1,AFC,-1.018501,5,11,323
26442,other,0,0,109,1184.64,1177.40,-1,UEFA,144,1069.75,1074.08,2,UEFA,-1.018501,5,11,323
